# Machine Learning Benchmarking for Claim Frequency

**Project:** PRISM – Predictive & Research-based Insurance Statistical Modeling

## Objective
To benchmark GLM performance against machine learning models and evaluate the trade-off between accuracy and interpretability.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_poisson_deviance

from xgboost import XGBRegressor


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
freq = pd.read_csv(
    "/content/drive/MyDrive/freMTPL2freq.csv"
)

freq = freq.rename(columns={
    "IDpol": "policy_id",
    "ClaimNb": "claim_count",
    "Exposure": "exposure",
    "Area": "area",
    "VehPower": "vehicle_power",
    "VehAge": "vehicle_age",
    "DrivAge": "driver_age",
    "BonusMalus": "bonus_malus",
    "VehBrand": "vehicle_brand",
    "VehGas": "vehicle_gas"
})


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_poisson_deviance


In [ ]:
X = freq[[
    "area", "vehicle_power", "vehicle_age", "driver_age",
    "bonus_malus", "vehicle_brand", "vehicle_gas"
]]

y = freq["claim_count"]
exposure = freq["exposure"]


In [ ]:
X_train, X_test, y_train, y_test, exp_train, exp_test = train_test_split(
    X, y, exposure, test_size=0.25, random_state=42
)


In [ ]:
categorical = ["area", "vehicle_brand", "vehicle_gas"]
numerical = ["vehicle_power", "vehicle_age", "driver_age", "bonus_malus"]

preprocess = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("num", "passthrough", numerical)
])


In [ ]:
xgb_model = Pipeline([
    ("prep", preprocess),
    ("model", XGBRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="count:poisson",
        random_state=42
    ))
])


In [ ]:
xgb_model.fit(X_train, y_train, model__sample_weight=exp_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['area', 'vehicle_brand',
                                                   'vehicle_gas']),
                                                 ('num', 'passthrough',
                                                  ['vehicle_power',
                                                   'vehicle_age', 'driver_age',
                                                   'bonus_malus'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_byt...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [ ]:
y_pred = xgb_model.predict(X_test)


In [ ]:
mean_poisson_deviance(y_test, y_pred, sample_weight=exp_test)


0.34510019639716993

In [ ]:
from sklearn.metrics import mean_poisson_deviance

# Load expected frequency from GLM
glm_pred = pd.read_csv(
    "/content/drive/MyDrive/expected_frequency.csv"
)

# Align with test set
glm_test = glm_pred.loc[X_test.index, "expected_frequency"]

mean_poisson_deviance(y_test, glm_test, sample_weight=exp_test)


0.3579508408841741

## ML vs GLM Comparison

XGBoost achieves a slightly lower mean Poisson deviance than the GLM, indicating marginally better predictive performance.  
However, the GLM is retained as the production model due to its interpretability, stability, and regulatory suitability.  
This demonstrates the trade-off between accuracy and explainability in insurance pricing.
